In [9]:
import sys
sys.path.insert(0, "/u/home/m/michaelw/project-ngarud/microbiome_evolution/py3.8/microbiome_evolution_SHALON/postprocessing_scripts/")

import config
import pandas as pd
import numpy as np

import subprocess

import pickle

import os

import itertools

import datetime

#MIDAS postprocessing scripts
from calculate_intersample_changes import *
import parse_midas_data
import diversity_utils
import core_gene_utils
import parse_patric

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

within_host_changes = True

%matplotlib inline



### Species

In [2]:
species_list_path = "/u/project/ngarud/Garud_lab/metagenomic_fastq_files/Shalon_2023/metadata/species_snps.txt"

with open(species_list_path) as f:
    species_list = [line.strip() for line in f if line.strip()]

In [3]:
species_list[:40]

['Acidaminococcus_intestini_54097',
 'Actinomyces_graevenitzii_58300',
 'Actinomyces_sp_57735',
 'Actinomyces_sp_62581',
 'Actinomyces_viscosus_57672',
 'Adlercreutzia_equolifaciens_60310',
 'Aggregatibacter_aphrophilus_58143',
 'Akkermansia_muciniphila_55290',
 'Alistipes_finegoldii_56071',
 'Alistipes_indistinctus_62207',
 'Alistipes_onderdonkii_55464',
 'Alistipes_putredinis_61533',
 'Alistipes_senegalensis_58364',
 'Alistipes_shahii_62199',
 'Anaerostipes_hadrus_55206',
 'Atopobium_parvulum_59960',
 'Atopobium_sp_59401',
 'Bacteroidales_bacterium_58650',
 'Bacteroides_caccae_53434',
 'Bacteroides_cellulosilyticus_58046',
 'Bacteroides_clarus_62282',
 'Bacteroides_coprocola_61586',
 'Bacteroides_eggerthii_54457',
 'Bacteroides_finegoldii_57739',
 'Bacteroides_fragilis_54507',
 'Bacteroides_intestinalis_61596',
 'Bacteroides_massiliensis_44749',
 'Bacteroides_ovatus_58035',
 'Bacteroides_pectinophilus_61619',
 'Bacteroides_plebeius_61623',
 'Bacteroides_salyersiae_54873',
 'Bacteroid

In [4]:
good_species = []

for species in species_list:
    haploid_samples = diversity_utils.calculate_haploid_samples(species)
    if len(haploid_samples) > 1:
        good_species.append(species)
    else:
        continue

# Sample dataframe

In [5]:
metadata_map = parse_midas_data.parse_sample_metadata_map()


In [7]:
metadata_map = parse_midas_data.parse_sample_metadata_map()
samples_df = pd.DataFrame(columns = ['accession_id', 'species', 'haploid', 'subject_id'])

accession_id_vec = []
species_vec = []
haploid_vec = []
subject_id_vec = []

counter = 0
no_of_species = len(species_list)
for species in species_list:
    counter += 1
    print("Processing " + species + " ({}/{})".format(counter, no_of_species))
    high_coverage_samples = diversity_utils.calculate_highcoverage_samples(species)
    haploid_samples = diversity_utils.calculate_haploid_samples(species)
    
    haploid_boolean_vec = [True if sample in haploid_samples else False for sample in high_coverage_samples]
    subject_id_vec_temp = [metadata_map[sample][0] for sample in high_coverage_samples]
    
    species_vec = species_vec + [species]*len(high_coverage_samples)
    accession_id_vec = accession_id_vec + list(high_coverage_samples)
    haploid_vec = haploid_vec + haploid_boolean_vec
    subject_id_vec = subject_id_vec + subject_id_vec_temp

samples_df['accession_id'] = accession_id_vec
samples_df['species'] = species_vec
samples_df['haploid'] = haploid_vec
samples_df['subject_id'] = subject_id_vec

    
    
    

Processing Acidaminococcus_intestini_54097 (1/200)
Processing Actinomyces_graevenitzii_58300 (2/200)
Processing Actinomyces_sp_57735 (3/200)
Processing Actinomyces_sp_62581 (4/200)
Processing Actinomyces_viscosus_57672 (5/200)
Processing Adlercreutzia_equolifaciens_60310 (6/200)
Processing Aggregatibacter_aphrophilus_58143 (7/200)
Processing Akkermansia_muciniphila_55290 (8/200)
Processing Alistipes_finegoldii_56071 (9/200)
Processing Alistipes_indistinctus_62207 (10/200)
Processing Alistipes_onderdonkii_55464 (11/200)


/tmp/ipykernel_33089/2722743874.py:17: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  haploid_boolean_vec = [True if sample in haploid_samples else False for sample in high_coverage_samples]


Processing Alistipes_putredinis_61533 (12/200)
Processing Alistipes_senegalensis_58364 (13/200)
Processing Alistipes_shahii_62199 (14/200)
Processing Anaerostipes_hadrus_55206 (15/200)
Processing Atopobium_parvulum_59960 (16/200)
Processing Atopobium_sp_59401 (17/200)
Processing Bacteroidales_bacterium_58650 (18/200)
Processing Bacteroides_caccae_53434 (19/200)
Processing Bacteroides_cellulosilyticus_58046 (20/200)
Processing Bacteroides_clarus_62282 (21/200)
Processing Bacteroides_coprocola_61586 (22/200)
Processing Bacteroides_eggerthii_54457 (23/200)
Processing Bacteroides_finegoldii_57739 (24/200)
Processing Bacteroides_fragilis_54507 (25/200)
Processing Bacteroides_intestinalis_61596 (26/200)
Processing Bacteroides_massiliensis_44749 (27/200)
Processing Bacteroides_ovatus_58035 (28/200)
Processing Bacteroides_pectinophilus_61619 (29/200)
Processing Bacteroides_plebeius_61623 (30/200)
Processing Bacteroides_salyersiae_54873 (31/200)
Processing Bacteroides_stercoris_56735 (32/200)
P

In [8]:
samples_df

,accession_id,species,haploid,subject_id
0,SRR18585109,Acidaminococcus_intestini_54097,True,14
1,SRR18585184,Acidaminococcus_intestini_54097,False,12
2,SRR18585205,Acidaminococcus_intestini_54097,True,9
3,SRR18585207,Acidaminococcus_intestini_54097,True,9
4,SRR18585211,Acidaminococcus_intestini_54097,True,9
...,...,...,...,...
1540,SRR18585105,Veillonella_sp_62404,False,14
1541,SRR18585106,Veillonella_sp_62404,False,14
1542,SRR18585124,Veillonella_sp_62404,False,13
1543,SRR18585145,Veillonella_sp_62404,False,5


### Plotting

In [ ]:
species_grouping = samples_df.groupby(['species', 'haploid']).size().reset_index(name='row_count')
subject_grouping = samples_df.groupby(['subject_id', 'haploid']).size().reset_index(name='row_count')

In [ ]:
species_grouping.groupby(by = ['species'])['row_count'].sum().reset_index().sort_values(by = 'row_count', ascending = False)#['species'].values


In [ ]:
species_grouping[species_grouping.species == "Bacteroides_vulgatus_57955"]

In [ ]:
fig, ax = plt.subplots(figsize=(40, 8))
sns.barplot(x = "species", y = "row_count", 
            data=species_grouping, 
            hue = "haploid", dodge = False, 
            ax = ax, 
            order = species_grouping.groupby(by = ['species'])['row_count'].sum().reset_index().sort_values(by = 'row_count', ascending = False)['species'].values

)
ax.set_xlabel("Species", fontsize=20)
ax.set_ylabel("Number of samples", fontsize=20)
ax.tick_params(axis='both', which='both', labelsize=20)
#ax.set_title("QP Samples", fontsize=30)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")



In [ ]:
figure_folder = "%s%s" % (config.analysis_directory, "figures/")
out_file = "%s%s" % (figure_folder, "QP_species_breakdown.png")
fig.savefig(out_file, dpi=300, bbox_inches='tight')

In [ ]:
figure_folder

In [ ]:
fig, ax = plt.subplots(figsize=(20, 8))
sns.barplot(x = "subject_id", y = "row_count", 
            data=subject_grouping, 
            hue = "haploid", dodge = False, 
            order = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15"],
            ax = ax)
ax.set_xlabel("Subject", fontsize=20)
ax.set_ylabel("Number of samples", fontsize=20)
ax.tick_params(axis='both', which='both', labelsize=20)
ax.set_title("QP Samples", fontsize=30)



In [ ]:
figure_folder = "%s%s" % (config.analysis_directory, "figures/")
out_file = "%s%s" % (figure_folder, "QP_subject_breakdown.png")
fig.savefig(out_file, dpi=300, bbox_inches='tight')

In [ ]:
len(samples_df.accession_id.unique())